In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Carregar o dataset
train_dataset = datasets.ImageFolder(root='/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/train', transform=transform)
val_dataset = datasets.ImageFolder(root='/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/val', transform=transform)

# Configurar os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Configuração do Modelo

In [6]:
import torch
from torchvision import models

# Instanciar o modelo MobileNetV2 pré-treinado
model = models.mobilenet_v2(pretrained=True)

# Ajustar para 2 classes
model.classifier[1] = torch.nn.Linear(model.last_channel, 2) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Treinando no dispositivo: {device}")
model = model.to(device)

Treinando no dispositivo: cuda


Configuração do Otimizador e Função de Perda

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Treino

In [8]:
num_epochs = 5000  # Número de épocas

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Época {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    # Validação
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

Época 1/5000, Loss: 0.1823
Val Loss: 0.3579, Val Accuracy: 0.8982
Época 2/5000, Loss: 0.1320
Val Loss: 0.1656, Val Accuracy: 0.9527
Época 3/5000, Loss: 0.0880
Val Loss: 0.2372, Val Accuracy: 0.9418
Época 4/5000, Loss: 0.0772
Val Loss: 0.2687, Val Accuracy: 0.9455
Época 5/5000, Loss: 0.0552
Val Loss: 0.2072, Val Accuracy: 0.9491
Época 6/5000, Loss: 0.0538
Val Loss: 0.2708, Val Accuracy: 0.9455
Época 7/5000, Loss: 0.0474
Val Loss: 0.1810, Val Accuracy: 0.9455
Época 8/5000, Loss: 0.0326
Val Loss: 0.2332, Val Accuracy: 0.9455
Época 9/5000, Loss: 0.0251
Val Loss: 0.3513, Val Accuracy: 0.9164
Época 10/5000, Loss: 0.0521
Val Loss: 0.3431, Val Accuracy: 0.9455
Época 11/5000, Loss: 0.0509
Val Loss: 0.2440, Val Accuracy: 0.9527
Época 12/5000, Loss: 0.0217
Val Loss: 0.2892, Val Accuracy: 0.9564
Época 13/5000, Loss: 0.0154
Val Loss: 0.3176, Val Accuracy: 0.9600
Época 14/5000, Loss: 0.0066
Val Loss: 0.3751, Val Accuracy: 0.9527
Época 15/5000, Loss: 0.0095
Val Loss: 0.5330, Val Accuracy: 0.9527
Époc

Salvar o Modelo

In [10]:
torch.save(model.state_dict(), '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/result_models/mobilenet_v2_melanoma.pth')